# Predictive Maintenance using enhanced CMAPSS dataset

## objective:
system that predicts when the engine is starting to wear out so that you can fix it early
we don't want a simplistic model, as real engines gradually degrade

project goals: 
- detect stage of degradation
- predicts how long it'll stay in that shape before it gets worse
- calculate a risk score
  

approach:
- **Clustering** to identify 5 health stages
- **Classification** to estimate current health
- **Regression** to predict time to next failure
- **Risk scoring** to trigger intelligent maintenance alerts


## dataset:
NASA CMAPSS: [Link](https://www.kaggle.com/datasets/behrad3d/nasa-cmaps)

## Phase 1 : data-preprocessing

In [ ]:
import pandas as pd
import numpy as np
import pickle
from sklearn.preprocessing import StandardScaler

In [ ]:
# Original CMAPSS column names
columns = ['unit_number','time_in_cycles','setting_1','setting_2','TRA','T2','T24','T30','T50','P2','P15','P30','Nf',
           'Nc','epr','Ps30','phi','NRf','NRc','BPR','farB','htBleed','Nf_dmd','PCNfR_dmd','W31','W32']

# Chosen degradation-sensitive features
selected_features = ['T24', 'T30', 'T50', 'P15', 'P30', 'Nf', 'Nc', 
                     'Ps30', 'phi', 'NRf', 'NRc', 'BPR', 'htBleed', 'W31', 'W32']

train_data_processed = []

# Preprocess each dataset (FD001–FD004)
for idx in range(1, 5):
    train_data = pd.read_csv(f"/kaggle/input/nasa-cmaps/CMaps/train_FD00{idx}.txt", sep=" ", header=None)
    
    # Drop trailing blank columns
    train_data.drop(columns=[26, 27], inplace=True)
    
    # Assign meaningful column names
    train_data.columns = columns
    
    # Keep only required columns
    train_data = train_data[['unit_number', 'time_in_cycles'] + selected_features]
    
    # Calculate normalized RUL (life ratio)
    train_data['RUL'] = train_data['time_in_cycles'] / train_data.groupby('unit_number')['time_in_cycles'].transform('max')
    
    train_data_processed.append(train_data)
    print(f"Processed train_FD00{idx}.txt, number of rows: {train_data.shape[0]}")

# Combine all datasets
all_data = pd.concat(train_data_processed, ignore_index=True)

In [ ]:
train_data_processed

In [ ]:
# ✅ Normalize sensor features
scaler = StandardScaler()
all_data[selected_features] = scaler.fit_transform(all_data[selected_features])

# ✅ Save scaler for future use
with open("scaler.pkl", "wb") as f:
    pickle.dump(scaler, f)

In [ ]:
# ✅ Save preprocessed dataset
all_data.to_pickle("preprocessed_FD001_004.pkl")

print("✅ Preprocessing complete. Saved to preprocessed_FD001_004.pkl")

## Phase 2 : clustering & degradation stage labeling

In [ ]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import seaborn as sns
import pickle

df = pd.read_pickle("preprocessed_FD001_004.pkl")

In [ ]:
selected_features = ['T24', 'T30', 'T50', 'P15', 'P30', 'Nf', 'Nc', 
                     'Ps30', 'phi', 'NRf', 'NRc', 'BPR', 'htBleed', 'W31', 'W32']

kmeans = KMeans(n_clusters=5, random_state=42, n_init=42)
df['raw_cluster'] = kmeans.fit_predict(df[selected_features])

In [ ]:
cluster_rul_means = df.groupby('raw_cluster')['RUL'].mean().sort_values(ascending=False)
cluster_to_stage = {cluster: i for i, cluster in enumerate(cluster_rul_means.index)}
df['degradation_stage'] = df['raw_cluster'].map(cluster_to_stage)
df.drop(columns=['raw_cluster'], inplace=True)

In [ ]:
stage_counts = df['degradation_stage'].value_counts().sort_index()
print("Degradation Stage Distribution:")
print(stage_counts)

sns.barplot(x=stage_counts.index, y=cluster_rul_means.values)
plt.xlabel("Degradation Stage")
plt.ylabel("Average RUL")
plt.title("Cluster-derived Degradation Stages")
plt.grid(True)
plt.show()

df.to_pickle("clustered_FD001_004.pkl")
print("✅ Clustering complete. Saved to clustered_FD001_004.pkl")

In [ ]:
import pandas as pd
import numpy as np
from sklearn.cluster import AgglomerativeClustering
from sklearn.model_selection import train_test_split

# Load dataset
df = pd.read_pickle("clustered_FD001_004.pkl")

# Features
selected_features = ['T24', 'T30', 'T50', 'P15', 'P30', 'Nf', 'Nc', 
                     'Ps30', 'phi', 'NRf', 'NRc', 'BPR', 'htBleed', 'W31', 'W32']

# Subsample: 5000 rows max (tunable)
df_sample = df.sample(n=5000, random_state=42).copy()

# Run Agglomerative Clustering on the sample
agglo = AgglomerativeClustering(n_clusters=5)
df_sample['agglo_cluster'] = agglo.fit_predict(df_sample[selected_features])

# Map clusters to stages based on average RUL
agglo_rul_means = df_sample.groupby('agglo_cluster')['RUL'].mean().sort_values(ascending=False)
agglo_stage_map = {cluster: i for i, cluster in enumerate(agglo_rul_means.index)}
df_sample['agglo_stage'] = df_sample['agglo_cluster'].map(agglo_stage_map)

# Show result
print("Agglomerative Stage Distribution (Sampled):")
print(df_sample['agglo_stage'].value_counts().sort_index())


## Phase 3 : classification and regression

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

# Train KNN on sampled clusters to generalize to full dataset
X_knn_train = df_sample[selected_features]
y_knn_train = df_sample['agglo_stage']

knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_knn_train, y_knn_train)

# Predict on full dataset
df['agglo_stage'] = knn.predict(df[selected_features])

print("Predicted Agglomerative Stage Distribution (Full Data):")
print(df['agglo_stage'].value_counts().sort_index())

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report

X_class = df[selected_features]
y_class = df['degradation_stage']

X_train_c, X_test_c, y_train_c, y_test_c = train_test_split(X_class, y_class, test_size=0.2, random_state=42)

In [ ]:
# Random Forest
clf = RandomForestClassifier(n_estimators=100, class_weight='balanced', random_state=42)
clf.fit(X_train_c, y_train_c)
y_pred_rf = clf.predict(X_test_c)
print("Random Forest Accuracy:", accuracy_score(y_test_c, y_pred_rf))
print(classification_report(y_test_c, y_pred_rf))

In [ ]:
# Logistic Regression
logreg = LogisticRegression(max_iter=1000, class_weight='balanced')
logreg.fit(X_train_c, y_train_c)
y_pred_log = logreg.predict(X_test_c)
print("Logistic Regression Accuracy:", accuracy_score(y_test_c, y_pred_log))
print(classification_report(y_test_c, y_pred_log))


In [ ]:
# SVC
svc = SVC(probability=True, class_weight='balanced')
svc.fit(X_train_c, y_train_c)
y_pred_svc = svc.predict(X_test_c)
print("SVC Accuracy:", accuracy_score(y_test_c, y_pred_svc))
print(classification_report(y_test_c, y_pred_svc))

In [ ]:
def compute_cycles_to_next_stage(df):
    """Computes how many cycles until the next degradation stage."""
    cycles_to_next = []

    for unit in df['unit_number'].unique():
        unit_df = df[df['unit_number'] == unit].reset_index(drop=True)
        n = len(unit_df)

        for i in range(n):
            current_stage = unit_df.loc[i, 'degradation_stage']
            future = unit_df.iloc[i+1:]
            next_stage = future[future['degradation_stage'] > current_stage]

            if not next_stage.empty:
                delta = next_stage.index[0] - i
            else:
                delta = 0  # already at final stage

            cycles_to_next.append(delta)

    return cycles_to_next

# Apply it
df['cycles_to_next_stage'] = compute_cycles_to_next_stage(df)


In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Ridge
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Filter for valid regression rows
regression_df = df[df['cycles_to_next_stage'] > 0].copy()
X_regression = regression_df[selected_features]
y_regression = regression_df['cycles_to_next_stage']

X_train_r, X_test_r, y_train_r, y_test_r = train_test_split(X_regression, y_regression, test_size=0.2, random_state=42)

models = {
    'Random Forest': RandomForestRegressor(n_estimators=100, random_state=42),
    'Ridge': Ridge(alpha=1.0),
    'SVR': SVR(kernel='rbf', C=1.0, epsilon=0.1)
}

results = {}


In [ ]:
from sklearn.base import clone
import time

results = {}

for name, base_model in models.items():
    print(f"\nTraining {name}…", flush=True)
    
    # Clone so we don't mutate the original in the dict
    model = clone(base_model)
    
    # If the model supports parallelism, switch it on
    if hasattr(model, "n_jobs"):
        model.set_params(n_jobs=-1)
    
    t0 = time.time()
    model.fit(X_train_r, y_train_r)
    y_pred = model.predict(X_test_r)
    elapsed = time.time() - t0
    
    # Metrics
    mse  = mean_squared_error(y_test_r, y_pred)
    rmse = np.sqrt(mse)
    mae  = mean_absolute_error(y_test_r, y_pred)
    r2   = r2_score(y_test_r, y_pred)
    
    results[name] = {
        "model": model, "predictions": y_pred,
        "MSE": mse, "RMSE": rmse, "MAE": mae, "R²": r2,
        "fit_time_s": elapsed
    }
    
    print(f"{name}: RMSE={rmse:.2f} | MAE={mae:.2f} | R²={r2:.2f} | "
          f"time={elapsed:.1f}s", flush=True)

best_model_name = max(results, key=lambda k: results[k]["R²"])
best_predictions = results[best_model_name]["predictions"]



Training Random Forest…
Random Forest: RMSE=75.19 | MAE=53.97 | R²=0.02 | time=93.3s

Training Ridge…
Ridge: RMSE=74.86 | MAE=53.85 | R²=0.03 | time=0.1s

Training SVR…


## Phase 4 : risk Score Computation + maintenance Alert Logic

In [56]:
# Risk Score Calculation
def calculate_risk_score(df, classifier, regressor):
    X = df[selected_features]
    stage_probs = classifier.predict_proba(X)
    df['failure_prob'] = stage_probs[:, 4]
    df['time_to_next_stage'] = regressor.predict(X)
    epsilon = 1e-6
    df['raw_risk_product'] = df['failure_prob'] * df['time_to_next_stage']
    df['urgency_score'] = df['failure_prob'] / (df['time_to_next_stage'] + epsilon)
    for col in ['raw_risk_product', 'urgency_score']:
        min_val, max_val = df[col].min(), df[col].max()
        df[f'normalized_{col}'] = (df[col] - min_val) / (max_val - min_val + epsilon)
    return df

In [57]:
# Optimal Threshold Finder
from sklearn.metrics import precision_recall_curve, auc

def find_optimal_threshold(df, true_label_col):
    df['imminent_failure'] = (df[true_label_col] <= 30).astype(int)
    precision, recall, thresholds = precision_recall_curve(df['imminent_failure'], df['normalized_urgency_score'])
    f1_scores = 2 * (precision * recall) / (precision + recall + 1e-6)
    best_idx = np.argmax(f1_scores)
    best_threshold = thresholds[best_idx]
    return best_threshold

In [58]:
# Alert System
def generate_alerts(df, threshold=0.7):
    df['alert_product'] = df['normalized_raw_risk_product'] > threshold
    df['alert_urgency'] = df['normalized_urgency_score'] > threshold
    df['alert_combined'] = df['alert_product'] & df['alert_urgency']
    return df

In [60]:
# Identify the best regression model by R²
best_model_name = max(results, key=lambda k: results[k]['R²'])
best_predictions = results[best_model_name]['predictions']

# Apply end-to-end
classifier = clf
regressor = results[best_model_name]['model']

df_with_risk = calculate_risk_score(df, classifier, regressor)
optimal_threshold = find_optimal_threshold(df_with_risk, 'time_to_next_stage')
df_with_alerts = generate_alerts(df_with_risk, threshold=optimal_threshold)

## Some Visualisations : 

## Sensor Trends by Degradation Stage

In [ ]:
def visualize_sensor_trends():
    """
    Visualize how key sensors change across different degradation stages
    """
    # Load the processed dataset
    df = pd.read_pickle("clustered_FD001_004.pkl")
    
    # Select a random engine unit for visualization
    unit = np.random.choice(df['unit_number'].unique())
    unit_data = df[df['unit_number'] == unit].sort_values('time_in_cycles')
    
    # Select key sensors that typically show degradation patterns
    key_sensors = ['T24', 'T30', 'P30', 'Nf', 'Ps30', 'phi', 'NRf', 'BPR', 'W31']
    
    # Create a multi-panel plot
    fig, axes = plt.subplots(len(key_sensors), 1, figsize=(12, 20), sharex=True)
    fig.suptitle(f'Sensor Readings Over Time for Engine Unit {unit}', fontsize=16)
    
    # Set a colormap for degradation stages
    cmap = plt.cm.get_cmap('RdYlGn_r', 5)
    stage_colors = [cmap(i) for i in range(5)]
    
    # Plot each sensor
    for i, sensor in enumerate(key_sensors):
        ax = axes[i]
        scatter = ax.scatter(
            unit_data['time_in_cycles'], 
            unit_data[sensor],
            c=unit_data['degradation_stage'], 
            cmap=plt.cm.get_cmap('RdYlGn_r', 5),
            alpha=0.7,
            s=30
        )
        
        # Add vertical lines where degradation stage changes
        stage_changes = unit_data.loc[unit_data['degradation_stage'].diff() != 0, 'time_in_cycles']
        for change in stage_changes:
            ax.axvline(x=change, color='gray', linestyle='--', alpha=0.5)
        
        ax.set_ylabel(sensor, fontsize=10)
        ax.grid(True, alpha=0.3)
        
        # Add trend line
        z = np.polyfit(unit_data['time_in_cycles'], unit_data[sensor], 1)
        p = np.poly1d(z)
        ax.plot(unit_data['time_in_cycles'], p(unit_data['time_in_cycles']), 
                "r--", alpha=0.8, linewidth=1)
    
    # Add a legend for degradation stages
    cbar = fig.colorbar(scatter, ax=axes.ravel().tolist(), pad=0.01, aspect=40)
    cbar.set_label('Degradation Stage', fontsize=12)
    cbar.set_ticks(np.arange(0.5, 5.5))
    cbar.set_ticklabels(['Stage 0', 'Stage 1', 'Stage 2', 'Stage 3', 'Stage 4'])
    
    axes[-1].set_xlabel('Operating Cycles', fontsize=12)
    plt.tight_layout()
    plt.subplots_adjust(top=0.95)
    plt.show()
    
    return fig

# Call the function
visualize_sensor_trends()

## risk score dashboard

In [ ]:
def visualize_risk_dashboard():
    """
    Create a risk score dashboard showing multiple engines and their degradation
    """
    # Load the data with risk scores
    df_with_alerts = pd.read_pickle("clustered_FD001_004.pkl")
    
    # Compute risk scores if they don't exist
    if 'normalized_urgency_score' not in df_with_alerts.columns:
        # If risk scores don't exist, create mock data for visualization
        print("Risk scores not found, creating mock data for visualization")
        df_with_alerts['normalized_urgency_score'] = df_with_alerts['RUL'] * -1 + 1  # Inverse of RUL
        df_with_alerts['normalized_urgency_score'] = (df_with_alerts['normalized_urgency_score'] - 
                                                     df_with_alerts['normalized_urgency_score'].min()) / \
                                                    (df_with_alerts['normalized_urgency_score'].max() - 
                                                     df_with_alerts['normalized_urgency_score'].min())
        df_with_alerts['alert_urgency'] = df_with_alerts['normalized_urgency_score'] > 0.7
    
    # Select a few engine units for the dashboard
    n_engines = 6
    selected_units = np.random.choice(df_with_alerts['unit_number'].unique(), 
                                     size=min(n_engines, len(df_with_alerts['unit_number'].unique())), 
                                     replace=False)
    
    # Create plotly subplot figure
    fig = make_subplots(rows=n_engines, cols=1, 
                        subplot_titles=[f"Engine Unit {unit}" for unit in selected_units],
                        shared_xaxes=True,
                        vertical_spacing=0.02)
    
    # Color scale for risk score
    colorscale = [[0, 'green'], [0.5, 'yellow'], [0.7, 'orange'], [1.0, 'red']]
    
    # Add traces for each engine
    for i, unit in enumerate(selected_units):
        unit_df = df_with_alerts[df_with_alerts['unit_number'] == unit].sort_values('time_in_cycles')
        
        # Main trace with risk score as color
        fig.add_trace(
            go.Scatter(
                x=unit_df['time_in_cycles'],
                y=unit_df['normalized_urgency_score'],
                mode='lines+markers',
                marker=dict(
                    size=8,
                    color=unit_df['normalized_urgency_score'],
                    colorscale=colorscale,
                    showscale=True if i == 0 else False,  # Only show colorbar for first subplot
                    colorbar=dict(title="Risk Score") if i == 0 else None
                ),
                line=dict(width=2),
                name=f"Unit {unit}"
            ),
            row=i+1, col=1
        )
        
        # Add alert markers
        if 'alert_urgency' in unit_df.columns:
            alerts = unit_df[unit_df['alert_urgency']]
            if not alerts.empty:
                fig.add_trace(
                    go.Scatter(
                        x=alerts['time_in_cycles'],
                        y=alerts['normalized_urgency_score'],
                        mode='markers',
                        marker=dict(
                            symbol='star',
                            size=12,
                            color='red',
                            line=dict(width=1, color='black')
                        ),
                        name="Alert"
                    ),
                    row=i+1, col=1
                )
        
        # Add threshold line
        if 'alert_urgency' in unit_df.columns:
            threshold = unit_df[unit_df['alert_urgency']]['normalized_urgency_score'].min() \
                        if not unit_df[unit_df['alert_urgency']].empty else 0.7
            fig.add_shape(
                type="line",
                x0=min(unit_df['time_in_cycles']),
                y0=threshold,
                x1=max(unit_df['time_in_cycles']),
                y1=threshold,
                line=dict(color="red", width=1, dash="dash"),
                row=i+1, col=1
            )
    
    # Update layout
    fig.update_layout(
        title_text="Engine Risk Score Dashboard",
        showlegend=False,
        height=200 * n_engines,
        width=1000,
        margin=dict(l=50, r=50, t=50, b=50)
    )
    
    # Update y-axes
    for i in range(1, n_engines + 1):
        fig.update_yaxes(title_text="Risk Score", range=[0, 1], row=i, col=1)
    
    # Update bottom x-axis
    fig.update_xaxes(title_text="Operating Cycles", row=n_engines, col=1)
    
    fig.show()
    return fig

# Call the function
visualize_risk_dashboard()